토치텍스트에서 배치퍼스트(batch_first)를 True로 한 경우와 False를 한 경우를 비교해보겠습니다.

text의 field에서는 batch_first를 True해줘야할 듯!!?

## 1. 훈련 데이터와 테스트 데이터로 분리하기

In [1]:
import urllib.request
import pandas as pd

#인터넷에서 IMDB 리뷰 데이터를 다운로드 받습니다.
urllib.request.urlretrieve("https://raw.githubusercontent.com/LawrenceDuan/IMDb-Review-Analysis/master/IMDb_Reviews.csv", filename="IMDb_Reviews.csv")

#다운로드한 IMDB 리뷰 데이터를 데이터프레임에 저장합니다.
df = pd.read_csv('IMDb_Reviews.csv', encoding='latin1')

print('전체 샘플의 개수 : {}'.format(len(df)))

#25,000개씩 분리하여 훈련 데이터와 테스트 데이터로 분리해보겠습니다.
train_df = df[:25000]
test_df = df[25000:]


train_df.to_csv("train_data.csv", index=False)
test_df.to_csv("test_data.csv", index=False) #index=False를 하면 인덱스를 저장하지 않습니다.

전체 샘플의 개수 : 50000


In [3]:
import os
print(os.getcwd())

/content


## 2. 필드 정의하기 (torchtext.legacy.data)

In [7]:
#필드 정의
from torchtext.legacy import data

# 필드 정의
TEXT = data.Field(sequential=True,
                  use_vocab=True,
                  tokenize=str.split,
                  lower=True,
                  batch_first=True, # <== 이 부분을 True로 합니다.
                  fix_length=20)

LABEL = data.Field(sequential=False,
                   use_vocab=False,
                   batch_first=False,
                   is_target=True)

* batch_first : 미니 배치 차원을 맨 앞으로 하여 데이터를 불러올 것인지 여부. (False가 기본값)

## 3. Dataset/Vocabulary/DataLoader 만들기

In [8]:
from torchtext.legacy.data import TabularDataset
from torchtext.legacy.data import Iterator

# TabularDataset은 데이터를 불러오면서 필드에서 정의했던 토큰화 방법으로 토큰화를 수행합니다.
train_data, test_data = TabularDataset.splits(
        path='.', train='train_data.csv', test='test_data.csv', format='csv',
        fields=[('text', TEXT), ('label', LABEL)], skip_header=True)

# 정의한 필드에 .build_vocab() 도구를 사용하면 단어 집합을 생성합니다.
TEXT.build_vocab(train_data, min_freq=10, max_size=10000) # 10,000개의 단어를 가진 단어 집합 생성

# 배치 크기를 정하고 첫번째 배치를 출력해보겠습니다.
batch_size = 5
train_loader = Iterator(dataset=train_data, batch_size = batch_size)
batch = next(iter(train_loader)) # 첫번째 미니배치

In [9]:
print(batch.text)

tensor([[6110,    0,    0,    7,    3,   87,  108,   95,  425,    0,   41,   87,
            0, 4409,    0,   87,  975,    4,  311,  299],
        [  60, 4501,   50, 4988,    4,  239,    0, 7163,   11,   14,   76,   33,
          194,  196,   11,   54,  328,   58,   20,    0],
        [  10,    7,    3,  410,  596,  841,    0,    3,  216,    5,    0,    0,
            3,  216,    5,    0,   36,  401, 2801,   60],
        [  44,  291,   94,    2,  151,   77,    5,  366,    0,    0,    0,    0,
          407,   28,    0,  668,   44,  291,   94,    3],
        [  44,   24,  336,  127,   37,  293,    0, 1544, 1518,  179, 1099,    0,
          895,   10, 1220, 6338,   29,   36, 4373,   11]])


배치 크기가 5이기 때문에 5개의 샘플이 출력되는 것을 볼 수 있습니다. 

각 샘플의 길이는 20의 길이를 가지는데, 이는 앞서 초기에 필드를 정의할 때 fix_length를 20으로 정해주었기 때문입니다. 

다시 말해 하나의 미니 배치의 크기는 (배치 크기 × fix_length)이다.

미니 배치의 크기를 출력해봅시다.

In [10]:
print(batch.text.shape)

torch.Size([5, 20])


**batch_first를 False로 한 경우를 보겠습니다.**

## 4. 필드 재정의하기

In [11]:
# 필드 정의
TEXT = data.Field(sequential=True,
                  use_vocab=True,
                  tokenize=str.split,
                  lower=True,
                  fix_length=20) 
#TEXT 필드에서 batch_first = True 인자를 제거하였습니다. 기본값이 False이므로 batch_first의 값이 False가 됩니다.

LABEL = data.Field(sequential=False,
                   use_vocab=False,
                   batch_first=False,
                   is_target=True)

필드를 재정의하였으므로 데이터셋, 단어 집합, 데이터로더를 만드는 과정을 다시 수행해주어야 합니다.

In [12]:
from torchtext.legacy.data import TabularDataset
from torchtext.legacy.data import Iterator

# TabularDataset은 데이터를 불러오면서 필드에서 정의했던 토큰화 방법으로 토큰화를 수행합니다.
train_data, test_data = TabularDataset.splits(
        path='.', train='train_data.csv', test='test_data.csv', format='csv',
        fields=[('text', TEXT), ('label', LABEL)], skip_header=True)

# 정의한 필드에 .build_vocab() 도구를 사용하면 단어 집합을 생성합니다.
TEXT.build_vocab(train_data, min_freq=10, max_size=10000) # 10,000개의 단어를 가진 단어 집합 생성

# 배치 크기를 정하고 첫번째 배치를 출력해보겠습니다.
batch_size = 5
train_loader = Iterator(dataset=train_data, batch_size = batch_size)
batch = next(iter(train_loader)) # 첫번째 미니배치

In [13]:
# 첫번째 미니 배치 출력
print(batch.text)

tensor([[6110,   60,   10,   44,   44],
        [   0, 4501,    7,  291,   24],
        [   0,   50,    3,   94,  336],
        [   7, 4988,  410,    2,  127],
        [   3,    4,  596,  151,   37],
        [  87,  239,  841,   77,  293],
        [ 108,    0,    0,    5,    0],
        [  95, 7163,    3,  366, 1544],
        [ 425,   11,  216,    0, 1518],
        [   0,   14,    5,    0,  179],
        [  41,   76,    0,    0, 1099],
        [  87,   33,    0,    0,    0],
        [   0,  194,    3,  407,  895],
        [4409,  196,  216,   28,   10],
        [   0,   11,    5,    0, 1220],
        [  87,   54,    0,  668, 6338],
        [ 975,  328,   36,   44,   29],
        [   4,   58,  401,  291,   36],
        [ 311,   20, 2801,   94, 4373],
        [ 299,    0,   60,    3,   11]])


하나의 미니 배치의 크기는 (fix_length × 배치 크기)입니다.

In [14]:
print(batch.text.shape)

torch.Size([20, 5])
